# Reto Cobify - PART 2

In [1]:
import pandas as pd
import numpy as np

import h2o
from h2o.automl import H2OAutoML
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

# 4. Regression models

Let's use h2o since it is a tool that automatically finds the most optimal model for the dataset given

### Initializing h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_271"; Java(TM) SE Runtime Environment (build 1.8.0_271-b09); Java HotSpot(TM) 64-Bit Server VM (build 25.271-b09, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/25/3rfj5zyn0cx18kcqkgd4nn4m0000gn/T/tmplglm_ecy
  JVM stdout: /var/folders/25/3rfj5zyn0cx18kcqkgd4nn4m0000gn/T/tmplglm_ecy/h2o_Angela_started_from_python.out
  JVM stderr: /var/folders/25/3rfj5zyn0cx18kcqkgd4nn4m0000gn/T/tmplglm_ecy/h2o_Angela_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,1 month and 30 days
H2O_cluster_name:,H2O_from_python_Angela_w18axj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


### Loading dataset

In [3]:
print('Loading dataset...')
train = h2o.import_file("../data/train_dummy.csv")
test = h2o.import_file("../data/test_dummy.csv")

Loading dataset...
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
x = train.columns # X contains the list of columns used for training
y = "consume" # Specifies the name of the column we want to predict
x.remove(y) # We delete the column "consume" from x

### Looking for the best algorithms...

In [6]:
automl = H2OAutoML(max_runtime_secs=1200)
automl.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [7]:
predictions = automl.predict(test)


stackedensemble prediction progress: |████████████████████████████████████| 100%


In [8]:
model = predictions.as_data_frame()
columns = ['Consume_Predicted']
model.columns = columns

In [9]:
model = model.round(2)
model

,Consume_Predicted
0,5.05
1,4.57
2,4.91
3,5.07
4,4.53
...,...
70,4.92
71,6.66
72,4.61
73,4.85


# 5. Metrics

### Checking for the model performance

In [10]:
lb = automl.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20210117_142714,0.412078,0.641933,0.412078,0.460037,0.100263
GBM_grid__1_AutoML_20210117_142714_model_6,0.419198,0.647455,0.419198,0.464038,0.100675
DeepLearning_grid__1_AutoML_20210117_142714_model_3,0.423319,0.650629,0.423319,0.477901,0.103334
DeepLearning_grid__1_AutoML_20210117_142714_model_1,0.436978,0.661043,0.436978,0.49985,0.105633
StackedEnsemble_AllModels_AutoML_20210117_142714,0.437584,0.661501,0.437584,0.462293,0.100911
DeepLearning_grid__1_AutoML_20210117_142714_model_6,0.445218,0.667247,0.445218,0.482374,0.104834
DeepLearning_grid__2_AutoML_20210117_142714_model_4,0.45551,0.674915,0.45551,0.502199,0.105489
XGBoost_3_AutoML_20210117_142714,0.455836,0.675156,0.455836,0.468394,0.104724
GBM_grid__1_AutoML_20210117_142714_model_14,0.463474,0.680789,0.463474,0.483587,0.104308
GBM_grid__1_AutoML_20210117_142714_model_15,0.464119,0.681263,0.464119,0.463053,0.104144


In [14]:
ld = automl.leader

In [15]:
m = h2o.get_model(ld)


Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20210117_142714

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.16718461572845397
RMSE: 0.4088821538395311
MAE: 0.31092996975764525
RMSLE: 0.06823636986167171
R^2: 0.8006564329953476
Mean Residual Deviance: 0.16718461572845397
Null degrees of freedom: 300
Residual degrees of freedom: 297
Null deviance: 252.44140099636846
Residual deviance: 50.32256933426465
AIC: 325.81536226797

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.41207774001366776
RMSE: 0.6419328158099318
MAE: 0.460036544851541
RMSLE: 0.10026256678240221
R^2: 0.5086566654496644
Mean Residual Deviance: 0.41207774001366776
Null degrees of freedom: 300
Residual degrees of freedom: 297
Null deviance: 258.29561534722313
Residual deviance: 124.03539974411399
AIC: 597.3514763075408
Model Details
H2OStackedEnsembl

H2OTypeError: Argument `model_id` should be a string, got H2OStackedEnsembleEstimator 

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20210117_142714

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.16718461572845397
RMSE: 0.4088821538395311
MAE: 0.31092996975764525
RMSLE: 0.06823636986167171
R^2: 0.8006564329953476
Mean Residual Deviance: 0.16718461572845397
Null degrees of freedom: 300
Residual degrees of freedom: 297
Null deviance: 252.44140099636846
Residual deviance: 50.32256933426465
AIC: 325.81536226797

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.41207774001366776
RMSE: 0.6419328158099318
MAE: 0.460036544851541
RMSLE: 0.10026256678240221
R^2: 0.5086566654496644
Mean Residual Deviance: 0.41207774001366776
Null degrees of freedom: 300
Residual degrees of freedom: 297
Null deviance: 258.29561534722313
Residual deviance: 124.03539974411399
AIC: 597.3514763075408


# 6. Conclusions

- H2O has chosen **Stacked Ensemble** has the best and most optimal model for the measurements dataset. Nevertheless, there are a few things worth mentioning:
    - We know that the smaller the RMSE, better the fit. However, it is also important to take into account the size of the dataset. In this case, our dataset has around 300 rows and 10 columns, so we cannot consider it a "big" dataset. Which is why, in this case, an RMSE of 0.40 is not bad at all.
    - Moreover, we have an R2 of 80% which indicates that the model explains accurately 80% of the dataset, which is a good metric.
    - It is also important to mention that the cross-validation metrics are quite different from the original ones (RMSE=0.40 vs. RMSE_cv=0.64 // R2=0.80 vs R2_cv=0.50). This indicates that even though the metrics found at first from the model are not random, they could be even more accurate.
- Going back to the analysis of the graphs from `1_wrangling`, we've seen than the gas type **SP98** allows **longer distances** and is **faster** than E10, however it **consumes more**. So if the company is looking to cut costs and spend less resources into fuel, E10 can be a good option to invest in when journeys are shorter and time is not a constraint

# Future Improvement

1. Enrich dataset either through web scraping or an API (ie: https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/)
2. Import dataset to a database such as SQL or MongoDB
3. Create an API to make the information in the dataset more accessible to others